In [1]:
import json
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from PIL import Image
import numpy as np

In [2]:
rawdata = json.load(open('modified_Data_1.json'))["data"]
liwc_ids = np.genfromtxt('liwc_Data.csv', delimiter=',',usecols=(0),dtype=None).tolist()
liwc_data = np.genfromtxt('liwc_Data_noID.csv', delimiter=',')

In [3]:
rawIds = list()
Y = list()
'''just removing "\xef\xbb\xbf" from the first ID'''
liwc_ids[0] = '966590693376781_1502441776458334'

In [4]:
'''there are 3371 rows in raw data and 2971 rows, 93 cols with messages (LIWC)
from raw data, we need dayofweek, month for X and likes for Y'''
picW = 128
picH = 128
numPixels = picW * picH
channels = 3
rawX = np.zeros((len(rawdata), 95 + picW*picH*channels))
print(rawX.shape)

(3371, 49247)


In [5]:
for i in range(len(rawdata)):
    postID = rawdata[i]['id']
    picFileName = rawdata[i]['picFileName']
    rawIds.append(postID)
    rawX[i][0] = rawdata[i]['dayofweek']
    rawX[i][1] = rawdata[i]['month']
    if(postID in liwc_ids) :
        rawX[i][2:95] = liwc_data[liwc_ids.index(postID)]
    if(picFileName != '') :
        img = Image.open('Resized Pics 128x128/'+picFileName).load()
        for imgX in range (picW):
            for imgY in range (picH):
                px = img[imgX, imgY]
                if type(px) is int : r = g = b = px
                elif len(px) is 3 : r, g, b = px
                else : r, g, b, _ = px
                baseIdx = 95 + imgX * picH + imgY
                rawX[i][baseIdx] = r
                rawX[i][baseIdx + numPixels] = g
                rawX[i][baseIdx + numPixels + numPixels] = b
    if(rawdata[i]['likes'] < 100):
        Y.append(0)
    else:
        Y.append(1)

In [6]:
print(len(rawIds))
print(rawX.shape)
print(len(Y))
print(rawX[:,:95].shape)

3371
(3371, 49247)
3371
(3371, 95)


In [7]:
'''split data'''
trainIDs, testIDs, Xtrain_nonimg, Xtest_nonimg, ytrain_nonimg, ytest_nonimg = train_test_split(rawIds, rawX[:,:95], Y, test_size=0.2, random_state=420)

trainIDs, testIDs, Xtrain_img, Xtest_img, ytrain_img, ytest_img = train_test_split(rawIds, rawX[:,95:], Y, test_size=0.2, random_state=420)

'''standardize and normalize'''
normalizer1 = Normalizer()
scaler1 = StandardScaler()

normalizer2 = Normalizer()
scaler2 = StandardScaler()

scaler1.fit(Xtrain_nonimg)
normalizer1.fit(Xtrain_nonimg)
scaler2.fit(Xtrain_img)
normalizer2.fit(Xtrain_img)

Xtrain_nonimg = scaler1.transform(Xtrain_nonimg)
Xtrain_nonimg = normalizer1.transform(Xtrain_nonimg)
Xtrain_img = scaler2.transform(Xtrain_img)
Xtrain_img = normalizer2.transform(Xtrain_img)

Xtest_nonimg = scaler1.transform(Xtest_nonimg)
Xtest_nonimg = normalizer1.transform(Xtest_nonimg)
Xtest_img = scaler2.transform(Xtest_img)
Xtest_img = normalizer2.transform(Xtest_img)

In [8]:
# Create classifiers
clf1 = MLPClassifier(solver='lbfgs', activation='logistic')
#clf1 = MLPClassifier(solver='lbfgs', activation='logistic', hidden_layer_sizes=(20000))
clf2 = MLPClassifier(solver='lbfgs', activation='logistic')

In [9]:
#Fit data
clf1.fit(Xtrain_nonimg, ytrain_nonimg)
clf2.fit(Xtrain_img, ytrain_img)

MLPClassifier(activation='logistic', alpha=0.0001, batch_size='auto',
       beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='lbfgs', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [17]:
#Predict
ypred_nonimg = clf1.predict(Xtest_nonimg)
ypred_img = clf2.predict(Xtest_img)

print(accuracy_score(ytest_img, ypred_img))
print(accuracy_score(ytest_nonimg, ypred_nonimg))

0.568888888889
0.602962962963


In [21]:
#Weight classifiers
eclf = VotingClassifier(estimators=[('mlp1', clf1), ('mlp2', clf2)], voting='soft', weights=[1, 1])

scaler1.fit(rawX)
normalizer1.fit(rawX)

y_pred = eclf.fit(rawX, Y)

In [22]:
#Test accuracy
print(accuracy_score(Y, y_pred))
print(classification_report(Y, y_pred))

TypeError: Expected sequence or array-like, got estimator VotingClassifier(estimators=[('mlp1', MLPClassifier(activation='logistic', alpha=0.0001, batch_size='auto',
       beta_1=0.9, beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_m...True, solver='lbfgs', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False))],
         flatten_transform=None, n_jobs=1, voting='soft', weights=[1, 1])